# Classification with Decision Tree and Naive Bayes example

### Importing MLlib libraries

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.classification import NaiveBayes
import pyspark.mllib.linalg
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark import SparkConf, SparkContext

### Read data and pre-processing
The **Iris flower data set** or Fisher's Iris data set is a multivariate data set introduced by Ronald Fisher in his 1936 paper "The use of multiple measurements in taxonomic problems as an example of linear discriminant analysis". The data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimetres.

In [ ]:
...
conf = SparkConf()
...
sc = SparkContext(conf=conf)

In [ ]:
rawData = sc.textFile("data/iris.csv")

In [ ]:
'''
val splitlines = rawData.map(lines => {
    lines.split(',')
  })
splitlines.first()
'''
splitlines = rawData.map(lambda lines: lines.split(','))
splitlines.first()

In [ ]:
'''
val Data = splitlines.map { col =>   
     val species = col(col.size - 1)                       
     val label = if (species == "Iris-versicolor") 0.toInt else if (species == "Iris-setosa") 1.toInt else 2.toInt
     val features = col.slice(0, col.size - 1).map(_.toDouble)
     LabeledPoint(label, Vectors.dense(features))
}
Data.take(5).foreach(println)
'''
def iris_label(argument):
    iris = {
        "Iris-versicolor": 0,
        "Iris-setosa": 1,
        "Iris-virginica": 2,
    }
    return iris.get(argument, 0)

def estractor(col):
    species = col[-1]
    label = iris_label(species)
    features = [float(x) for x in col[:-1]]
    return LabeledPoint(label, Vectors.dense(features))
    
Data = splitlines.map(estractor)

for x in Data.take(5):
    print(x)

### Split the data into training and test sets (40% held out for testing)

In [ ]:
'''
val splits = Data.randomSplit(Array(0.6, 0.4), seed = 11L)
val trainingData = splits(0).cache()
val testData = splits(1)
println("Training Data")
trainingData.take(5).foreach(println)
println("Test Data")
testData.take(5).foreach(println)
'''
splits = Data.randomSplit([0.6, 0.4], seed = 11)
trainingData = splits[0].cache()
testData = splits[1]
print(trainingData.count())
print("Training Data")
for x in trainingData.take(5):
    print(x)
print("Test Data")
for x in testData.take(5):
    print(x)

### Train a Decision Tree
Decision trees are widely used since they are easy to interpret, handle categorical features, extend to the multiclass classification setting, do not require feature scaling and are able to capture nonlinearities and feature interactions. Tree ensemble algorithms such as random forests and boosting are among the top performers for classification and regression tasks. MLlib supports decision trees for binary and multiclass classification and for regression, using both continuous and categorical features. The implementation partitions data by rows, allowing distributed training with millions of instances.

In [ ]:
'''
val numClasses = 3
val categoricalFeaturesInfo = Map[Int, Int]()
val impurity = "entropy"
val maxDepth = 3
val maxBins = 10
val dtModel = DecisionTree.trainClassifier(trainingData, numClasses, categoricalFeaturesInfo,
  impurity, maxDepth, maxBins)
println(dtModel.toDebugString)
'''
numClasses = 3
categoricalFeaturesInfo = {} #Map[Int, Int]()
impurity = "entropy"
maxDepth = 3
maxBins = 10
dtModel = DecisionTree.trainClassifier(trainingData, numClasses, categoricalFeaturesInfo, impurity, maxDepth, maxBins)
print(dtModel.toDebugString())

In [ ]:
'''
val dtTotalCorrect = trainingData.map { point =>
  if (dtModel.predict(point.features) == point.label) 1 else 0
  }.sum

println(dtTotalCorrect)
println(trainingData.count)
'''
predictions = dtModel.predict(trainingData.map(lambda point: point.features))
labelsAndPredictions = trainingData.map(lambda lp: lp.label).zip(predictions)
dtTotalCorrect = labelsAndPredictions.map(lambda line: 1 if line[0]==line[1] else 0).sum()

'''
for x in trainingData.collect():
    print(dtModel.predict(x.features))
'''

print(dtTotalCorrect)
print(trainingData.count())

In [ ]:
'''
val dtAccuracy = dtTotalCorrect / trainingData.count
println(dtAccuracy)
'''
dtAccuracy = dtTotalCorrect / trainingData.count()
print(dtAccuracy)

In [ ]:
'''
val dtTotalCorrect = testData.map { point =>
    if (nbModel.predict(point.features) == point.label) 1 else 0
}.sum

println(dtTotalCorrect)
println(testData.count)
'''
predictions = dtModel.predict(testData.map(lambda point: point.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
dtTotalCorrect = labelsAndPredictions.map(lambda line: 1 if line[0]==line[1] else 0).sum()

print(dtTotalCorrect)
print(testData.count())

In [ ]:
'''
val nbAccuracy = nbTotalCorrect / testData.count
println(nbAccuracy)
'''
dtAccuracy = dtTotalCorrect / testData.count()
print(dtAccuracy)

### Train Naive Bayes
Naive Bayes is a simple multiclass classification algorithm with the assumption of independence between every pair of features. Naive Bayes can be trained very efficiently. Within a single pass to the training data, it computes the conditional probability distribution of each feature given label, and then it applies Bayes theorem to compute the conditional probability distribution of label given an observation and use it for prediction.

In [ ]:
'''
val nbModel = NaiveBayes.train(trainingData)
println(nbModel)
'''
nbModel = NaiveBayes.train(trainingData,1.0)
print(nbModel)

In [ ]:
'''
val nbTotalCorrect = trainingData.map { point =>
    if (nbModel.predict(point.features) == point.label) 1 else 0
}.sum
println(nbTotalCorrect)
println(trainingData.count)
'''
predictions = nbModel.predict(trainingData.map(lambda point: point.features))
labelsAndPredictions = trainingData.map(lambda lp: lp.label).zip(predictions)
nbTotalCorrect = labelsAndPredictions.map(lambda line: 1 if line[0]==line[1] else 0).sum()
print(nbTotalCorrect)
print(trainingData.count())

In [ ]:
'''
val nbAccuracy = nbTotalCorrect / trainingData.count
println(nbAccuracy)
'''
nbAccuracy = nbTotalCorrect / trainingData.count()
print(nbAccuracy)

### Test

In [ ]:
'''
val nbTotalCorrect = testData.map { point =>
    if (nbModel.predict(point.features) == point.label) 1 else 0
}.sum

println(nbTotalCorrect)
println(testData.count)
'''
predictions = nbModel.predict(testData.map(lambda point: point.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
nbTotalCorrect = labelsAndPredictions.map(lambda line: 1 if line[0]==line[1] else 0).sum()
print(nbTotalCorrect)
print(testData.count())

In [ ]:
'''
val nbAccuracy = nbTotalCorrect / testData.count
println(nbAccuracy)
'''
nbAccuracy = nbTotalCorrect / testData.count()
print(nbAccuracy)

### Complete evaluation on test set for Decision Tree model

In [ ]:
'''
val predictionAndLabels = testData.map { case LabeledPoint(label, features) =>
  val prediction = dtModel.predict(features)
  (prediction, label)
}
'''
predictions = dtModel.predict(testData.map(lambda point: point.features))
predictionAndLabels = predictions.zip(testData.map(lambda lp: lp.label))

'''
// Instantiate metrics object
val metrics = new MulticlassMetrics(predictionAndLabels)
'''
metrics = MulticlassMetrics(predictionAndLabels)

'''
// Confusion matrix
println("Confusion matrix:")
println(metrics.confusionMatrix)
'''
print("Confusion matrix:")
#print(metrics.confusionMatrix())

'''
// Overall Statistics
val precision = metrics.precision
val recall = metrics.recall // same as true positive rate
val f1Score = metrics.fMeasure
println("Summary Statistics")
println(s"Precision = $precision")
println(s"Recall = $recall")
println(s"F1 Score = $f1Score")
'''
precision = metrics.precision()
recall = metrics.recall() # same as true positive rate
f1Score = metrics.fMeasure()
print("Summary Statistics")
print("Precision = ", precision)
print("Recall = ", recall)
print("F1 Score = ", f1Score)

'''
// Precision by label
val labels = metrics.labels
labels.foreach { l =>
    println(s"Precision($l) = " + metrics.precision(l))
}
'''
labels = trainingData.map(lambda l:l.label).distinct().sortBy(lambda l: l).collect()
for l in labels:
    print("Precision(%s) = %s" % (l,metrics.precision(l)))
    
'''
// Recall by label
labels.foreach { l =>
    println(s"Recall($l) = " + metrics.recall(l))
}
'''
for l in labels:
    print("Recall(%s) = %s" % (l,metrics.recall(l)))

'''
// False positive rate by label
labels.foreach { l =>
    println(s"FPR($l) = " + metrics.falsePositiveRate(l))
}
'''
for l in labels:
    print("FPR(%s) = %s" % (l,metrics.falsePositiveRate(l)))

'''
// F-measure by label
labels.foreach { l =>
    println(s"F1-Score($l) = " + metrics.fMeasure(l))
}
'''
for l in labels:
    print("F1-Score(%s) = %s" % (l,metrics.fMeasure(l)))

'''
// Weighted stats
println(s"Weighted precision: ${metrics.weightedPrecision}")
println(s"Weighted recall: ${metrics.weightedRecall}")
println(s"Weighted F1 score: ${metrics.weightedFMeasure}")
println(s"Weighted false positive rate: ${metrics.weightedFalsePositiveRate}")
'''
print("Weighted precision: ", metrics.weightedPrecision)
print("Weighted recall: ", metrics.weightedRecall)
print("Weighted F1 score: ", metrics.weightedFMeasure())
print("Weighted false positive rate: ", metrics.weightedFalsePositiveRate)

http://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#label-based-metrics

### Complete evaluation on test set for Naive Bayes model

In [ ]:
'''
val predictionAndLabels = testData.map { case LabeledPoint(label, features) =>
  val prediction = nbModel.predict(features)
  (prediction, label)
}
'''
predictions = nbModel.predict(testData.map(lambda point: point.features)).map(lambda pre: float(pre))
predictionAndLabels = predictions.zip(testData.map(lambda lp: lp.label))

'''
// Instantiate metrics object
val metrics = new MulticlassMetrics(predictionAndLabels)
'''
metrics = MulticlassMetrics(predictionAndLabels)

'''
// Confusion matrix
println("Confusion matrix:")
println(metrics.confusionMatrix)
'''
print("Confusion matrix:")
#print(metrics.confusionMatrix())

'''
// Overall Statistics
val precision = metrics.precision
val recall = metrics.recall // same as true positive rate
val f1Score = metrics.fMeasure
println("Summary Statistics")
println(s"Precision = $precision")
println(s"Recall = $recall")
println(s"F1 Score = $f1Score")
'''
precision = metrics.precision()
recall = metrics.recall() # same as true positive rate
f1Score = metrics.fMeasure()
print("Summary Statistics")
print("Precision = ", precision)
print("Recall = ", recall)
print("F1 Score = ", f1Score)

'''
// Precision by label
val labels = metrics.labels
labels.foreach { l =>
    println(s"Precision($l) = " + metrics.precision(l))
}
'''
labels = trainingData.map(lambda l:l.label).distinct().sortBy(lambda l: l).collect()
for l in labels:
    print("Precision(%s) = %s" % (l,metrics.precision(l)))
    
'''
// Recall by label
labels.foreach { l =>
    println(s"Recall($l) = " + metrics.recall(l))
}
'''
for l in labels:
    print("Recall(%s) = %s" % (l,metrics.recall(l)))

'''
// False positive rate by label
labels.foreach { l =>
    println(s"FPR($l) = " + metrics.falsePositiveRate(l))
}
'''
for l in labels:
    print("FPR(%s) = %s" % (l,metrics.falsePositiveRate(l)))

'''
// F-measure by label
labels.foreach { l =>
    println(s"F1-Score($l) = " + metrics.fMeasure(l))
}
'''
for l in labels:
    print("F1-Score(%s) = %s" % (l,metrics.fMeasure(l)))

'''
// Weighted stats
println(s"Weighted precision: ${metrics.weightedPrecision}")
println(s"Weighted recall: ${metrics.weightedRecall}")
println(s"Weighted F1 score: ${metrics.weightedFMeasure}")
println(s"Weighted false positive rate: ${metrics.weightedFalsePositiveRate}")
'''
print("Weighted precision: ", metrics.weightedPrecision)
print("Weighted recall: ", metrics.weightedRecall)
print("Weighted F1 score: ", metrics.weightedFMeasure())
print("Weighted false positive rate: ", metrics.weightedFalsePositiveRate)